In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math

## Question 1.5

#### Pretraining the network to obtain weights

In [31]:

"""
Heere we define network parametes and build the computational graph
"""

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

##network params
learning_rate = tf.constant(0.0003)
numEpochs = 400
batchSize = 50
displayStep = 5
nodesInEachHiddenLayer = [1024,1024,1024,1024,1024]  #list of nodes  excpet x and y

numExamples,xDim = mnist.train.images.shape
numExamples,yDim = mnist.train.labels.shape


totalNodes = nodesInEachHiddenLayer
X = tf.placeholder(tf.float32,[None,xDim],name = "X")
Y  =  tf.placeholder(tf.float32,[None,yDim], name = "Y")



## collecting weights in a dictionary



def defineWeightsAndBias(nodesInEachHiddenLayer,xDim, yDim):
  
  ## he inisitialisation
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  
  weights = [tf.Variable(initialiser([j,k],dtype=tf.float32)) \
             for (j,k) in zip([xDim]+nodesInEachHiddenLayer,nodesInEachHiddenLayer+[yDim])]
  
  
  biases = [tf.Variable(initialiser([j],dtype=tf.float32))
           for j in nodesInEachHiddenLayer+[yDim]]

#   weights = {'w'+str(i+1): tf.Variable(tf.random_normal([j,k],dtype=tf.float32)) \
#              for (i,(j,k)) in enumerate(zip([xDim]+nodesInEachHiddenLayer,nodesInEachHiddenLayer+[yDim]))}

#   biases = {'b'+str(i+1): tf.Variable(tf.random_normal([j],dtype=tf.float32))
#            for i,j in enumerate(nodesInEachHiddenLayer+[yDim])}

  return weights, biases



def getNetworkOutput(xInp,weights,biases):
  
    totalLayers = nodesInEachHiddenLayer+[yDim]
    layerOutputList = [0 for i in  range(len(totalLayers))]
    
    lastLayerPosition = len(totalLayers)-1
    
    for layerPosition,(weight,bias) in enumerate(zip(weights,biases)):

      if layerPosition == 0:
      
        layerOutputList[layerPosition] = tf.nn.relu(tf.add(tf.matmul(xInp, weight), bias))
         


      elif layerPosition <= lastLayerPosition-1:
      
        layerOutputList[layerPosition] = tf.nn.relu(tf.add(tf.matmul(layerOutputList[layerPosition-1], weight), bias))
        
      
      else:
        
        layerOutputList[layerPosition] = tf.add(tf.matmul(layerOutputList[layerPosition-1], weight), bias)
#         layerOut = tf.nn.relu(layerOut,name='activation_relu')
      
    
    return layerOutputList[-1],layerOutputList
  




Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
##call by functions

weights,biases = defineWeightsAndBias(nodesInEachHiddenLayer,xDim, yDim)
layer_Output, AllLayerOutputs = getNetworkOutput(X,weights,biases)

In [0]:
##loss calculations


lossCalcu  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_Output, labels=Y))

gradOptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train = gradOptimizer.minimize(lossCalcu)

## required Layers tensors
predictions = tf.nn.softmax(AllLayerOutputs[-1])

correct_pred = tf.equal(tf.argmax(layer_Output, 1), tf.argmax(Y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
initialise = tf.global_variables_initializer()

In [34]:
currSess = tf.InteractiveSession()




x_train = mnist.train.images

y_train = mnist.train.labels

##training 
currSess.run(initialise)

x_test = mnist.test.images
y_test = mnist.test.labels

for epoch in range(200):
  for iteration in range(mnist.train.num_examples // batchSize):
    xBatch, yBatch = mnist.train.next_batch(batchSize)  


    currSess.run(train,feed_dict ={X:xBatch,Y:yBatch})


  loss, acc = currSess.run([lossCalcu, accuracy], feed_dict={X: x_train, Y: y_train})  


  print("Step " + str(epoch) + ", Loss= " + str(loss) + ", Training Accuracy= " + str(acc))


  testAccuracy = currSess.run(accuracy, feed_dict={X: x_test,
                                Y:y_test })
  print("Testing Accuracy:",testAccuracy)
###printing the images for q3

  if testAccuracy >= 0.98:
    break


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step 0, Loss= 0.08887421, Training Accuracy= 0.9734727
Testing Accuracy: 0.9642
Step 1, Loss= 0.05022041, Training Accuracy= 0.9850182
Testing Accuracy: 0.9697
Step 2, Loss= 0.042661045, Training Accuracy= 0.98752725
Testing Accuracy: 0.9715
Step 3, Loss= 0.02873116, Training Accuracy= 0.99163634
Testing Accuracy: 0.9743
Step 4, Loss= 0.03629114, Training Accuracy= 0.9899455
Testing Accuracy: 0.9742
Step 5, Loss= 0.0261305, Training Accuracy= 0.99236363
Testing Accuracy: 0.9758
Step 6, Loss= 0.016531412, Training Accuracy= 0.9947091
Testing Accuracy: 0.9793
Step 7, Loss= 0.012913459, Training Accuracy= 0.9961454
Testing Accuracy: 0.981


#### get SVD approximated weights from original weight matrices

In [0]:
def SVDWeights(weightsList,topN = None):
  approxWeightsList = []
  lastPosition = len(weightsList)-1
  for position,weight in enumerate(weightsList):

    if position != lastPosition:
      
      
      if topN is not None:
        s, u, v = tf.linalg.svd(weight,full_matrices=True)
        approxWeightsList.append(tf.matmul(u[:,:topN], tf.matmul(tf.linalg.diag(s[:topN]), v[:,:topN], adjoint_b=True)))
        
      elif topN is None:
        s, u, v = tf.linalg.svd(weight,full_matrices=False)
        approxWeightsList.append(tf.matmul(u, tf.matmul(tf.linalg.diag(s), v, adjoint_b=True)))
    
    elif position == lastPosition:
       approxWeightsList.append(weightsList[-1])
    
  return approxWeightsList

# layer_Output, AllLayerOutputs = getNetworkOutput(X,weights,biases)




#### Feed forward with different D's 

In [7]:

for D in [10, 20, 50, 100, 200,None]:
  approxWeightsList = SVDWeights(weights,D)
  layer_Output, AllLayerOutputs = getNetworkOutput(X,approxWeightsList,biases)
  predictions = tf.nn.softmax(AllLayerOutputs[-1])

  correct_pred = tf.equal(tf.argmax(layer_Output, 1), tf.argmax(Y, 1))

  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  testAccuracy = currSess.run(accuracy, feed_dict={X: x_test,
                                Y:y_test })
  print(" D is : ", D ," and Testing Accuracy is : ",testAccuracy)

 D is :  10  and Testing Accuracy is :  0.6915
 D is :  20  and Testing Accuracy is :  0.9424
 D is :  50  and Testing Accuracy is :  0.9731
 D is :  100  and Testing Accuracy is :  0.9778
 D is :  200  and Testing Accuracy is :  0.9784
 D is :  None  and Testing Accuracy is :  0.9802


#### Storing all the variables that were previously initialised (weight matrices specifically so that they are not re initialised)

In [0]:
varsAlreadyInititalised = set(tf.all_variables())

#### Freezing weights in numpy and converting them non trainable

In [0]:
weights = [weight.eval() for weight in weights]


## 1.6 a

#### Decomposing weights into U and Vhat with D = 20 and storing them in a list 

In [0]:

def decomposeWeights(weightsList,topN = None):
  
  decomposedWeightsList = []
  
  lastPosition = len(weightsList)-1
  for position,weight in enumerate(weightsList):

    if position != lastPosition:
      
      
      if topN is not None:
        s, u, v = tf.linalg.svd(weight,full_matrices=True)
        
        vHat = tf.Variable(tf.matmul(tf.linalg.diag(s[:topN]), v[:,:topN], adjoint_b=True))
        
        u = tf.Variable(u[:,:topN])
        
        ###u and vhat are put into a tuple
        decomposedWeightsList.append((u, vHat))
        
    
    elif position == lastPosition:
       decomposedWeightsList.append(weightsList[-1])
    
  return decomposedWeightsList

decomposedWeightsList = decomposeWeights(weights,topN = 20)

#### network out put with SVD decomposition takes in decompedWeightsList as one of the argument

In [0]:

def getNetworkWithSVDOutput(xInp,decomposedWeightsList,biases):
  
    totalLayers = nodesInEachHiddenLayer+[yDim]
    
    
    lastLayerPosition = len(totalLayers)-1
    
    for layerPosition,(weight,bias) in enumerate(zip(decomposedWeightsList,biases)):

      if layerPosition == 0:
        
        ## u * vhat:  weight [0] contains u and weight[1] contains vhat
        approxWeight = tf.matmul(weight[0], weight[1])
        
        output = tf.nn.relu(tf.add(tf.matmul(xInp, approxWeight), bias))
        


      elif layerPosition <= lastLayerPosition-1:
        
        approxWeight = tf.matmul(weight[0], weight[1])
        output = tf.nn.relu(tf.add(tf.matmul(output, approxWeight), bias))
        
      
      else:
        
        output= tf.add(tf.matmul(output, weight), bias)
#         layerOut = tf.nn.relu(layerOut,name='activation_relu')
      
    
    return output

#### defining the rest of the trainng and accuracy calls 

getNetworkWithSVDOutput(X,decomposedWeightsList,biases) is called here

In [0]:
###output with new SVD decomposition

layerOutputSVD = getNetworkWithSVDOutput(X,decomposedWeightsList,biases)
lossCalcu  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=layerOutputSVD, labels=Y))

gradOptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train = gradOptimizer.minimize(lossCalcu)

## required Layers tensors
predictions = tf.nn.softmax(layerOutputSVD)

correct_pred = tf.equal(tf.argmax(layerOutputSVD, 1), tf.argmax(Y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

###initialising only those variable that were not initialised previously so that the weight matrices are not reinitialised
currSess.run(tf.variables_initializer(set(tf.all_variables()) - varsAlreadyInititalised))


#### Training procedure as usual

In [40]:

##training 
x_train = mnist.train.images

y_train = mnist.train.labels



x_test = mnist.test.images
y_test = mnist.test.labels

for epoch in range(200):
  for iteration in range(mnist.train.num_examples // batchSize):
    xBatch, yBatch = mnist.train.next_batch(batchSize)  


    currSess.run(train,feed_dict ={X:xBatch,Y:yBatch})


  loss, acc = currSess.run([lossCalcu, accuracy], feed_dict={X: x_train, Y: y_train})  


  print("Step " + str(epoch) + ", Loss= " + str(loss) + ", Training Accuracy= " + str(acc))


  testAccuracy = currSess.run(accuracy, feed_dict={X: x_test,
                                Y:y_test })
  print("Testing Accuracy:",testAccuracy)
###printing the images for q3

  if testAccuracy >= 0.97:
    break


Step 0, Loss= 0.051006038, Training Accuracy= 0.98414546
Testing Accuracy: 0.9747
